In [ ]:
# RwandanFoodAI Project Notebook

# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics.pairwise import cosine_similarity
import cv2


In [ ]:
# Set up paths
BASE_DIR = "RwandanFoodAI"
DATA_DIR = os.path.join(BASE_DIR, "Dataset", "Rwandan dishes")
NUTRITION_DATA_PATH = os.path.join(BASE_DIR, "data", "nutrition", "rwandan_food_data.csv")
MODEL_PATH = os.path.join(BASE_DIR, "models", "rwandan_food_model.h5")

In [ ]:
# Data Preprocessing
def preprocess_data(data_dir, target_size=(299, 299), batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )
    
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    
    validation_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, validation_generator

# Create and train the model
def create_model(num_classes):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def train_model(model, train_generator, validation_generator, epochs=50):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
    )
    
    return history

# Calorie Estimation
def estimate_portion_size(image_path, reference_object_size):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    food_contour = max(contours, key=cv2.contourArea)
    food_area = cv2.contourArea(food_contour)
    pixels_per_inch = np.sqrt(food_area / reference_object_size)
    food_size_inches = food_area / (pixels_per_inch ** 2)
    return food_size_inches

def estimate_calories(food_name, portion_size):
    calories_per_inch = {
        'ugali': 100,
        'isombe': 50,
        'matoke': 80,
        # Add more Rwandan dishes here
    }
    return calories_per_inch.get(food_name, 0) * portion_size

# Food Recommender
class FoodRecommender:
    def __init__(self, food_data):
        self.food_data = food_data
        self.similarity_matrix = self._compute_similarity()
    
    def _compute_similarity(self):
        return cosine_similarity(self.food_data.drop('name', axis=1))
    
    def get_recommendations(self, food_name, top_n=5):
        food_index = self.food_data[self.food_data['name'] == food_name].index[0]
        similar_foods = list(enumerate(self.similarity_matrix[food_index]))
        similar_foods = sorted(similar_foods, key=lambda x: x[1], reverse=True)
        
        recommendations = []
        for i in range(1, top_n + 1):
            recommendations.append(self.food_data.iloc[similar_foods[i][0]]['name'])
        
        return recommendations

# Main execution
if __name__ == "__main__":
    # Data preprocessing
    train_generator, validation_generator = preprocess_data(DATA_DIR)
    
    # Model training
    num_classes = len(train_generator.class_indices)
    model = create_model(num_classes)
    history = train_model(model, train_generator, validation_generator)
    
    # Save the model
    model.save(MODEL_PATH)
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Example usage
    food_name = 'Ugali'  # Example dish
    sample_image_path = os.path.join(DATA_DIR, food_name, os.listdir(os.path.join(DATA_DIR, food_name))[0])
    
    # Calorie estimation
    portion_size = estimate_portion_size(sample_image_path, reference_object_size=1)
    calories = estimate_calories(food_name, portion_size)
    print(f"Estimated calories for {food_name}: {calories}")
    
    # Recommendation
    food_data = pd.read_csv(NUTRITION_DATA_PATH)
    recommender = FoodRecommender(food_data)
    recommendations = recommender.get_recommendations(food_name)
    print(f"Recommendations for {food_name}: {recommendations}")

    # Visualize a sample image
    img = plt.imread(sample_image_path)
    plt.imshow(img)
    plt.title(f"Sample Image: {food_name}")
    plt.axis('off')
    plt.show()